In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from modules.constants import AWSConfig, TorchParams

In [3]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name
training_data_uri = "s3://{}/".format(AWSConfig.bucket)
metric_definitions = [
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ]


In [6]:
estimator = PyTorch(entry_point='speach_emotion_recognition_vit.py',
                            hyperparameters=TorchParams.hyper_param,
                            role=role,
                            source_dir=".",
                            framework_version='1.1',
                            py_version='py3',
                            instance_count=2,
                            instance_type='ml.c5.xlarge')

In [7]:
estimator.fit(training_data_uri, logs=True)

2022-08-03 17:05:24 Starting - Starting the training job...
2022-08-03 17:05:47 Starting - Preparing the instances for trainingProfilerReport-1659546324: InProgress
......
2022-08-03 17:06:51 Downloading - Downloading input data...
2022-08-03 17:07:22 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-03 17:07:24,349 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2022-08-03 17:07:24,351 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-08-03 17:07:24,364 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-03 17:07:24,365 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-03 17:07:24,650 sagemaker-containers INFO     Module speach_emotion_recognition_vit does not provide a setup.py. 
Generating setup.py
20

UnexpectedStatusException: Error for Training job sagemaker-pytorch-2022-08-03-17-05-24-101: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python -m speach_emotion_recognition_vit --batch-size 64 --epochs 50 --lr 3e-05 --momentum 0.7"
Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/ml/code/speach_emotion_recognition_vit.py", line 8, in <module>
    import timm
  File "/usr/local/lib/python3.6/dist-packages/timm/__init__.py", line 2, in <module>
    from .models import create_model, list_models, is_model, list_modules, model_entrypoint, \
  File "/usr/local/lib/python3.6/dist-packages/timm/models/__init__.py", line 1, in <module>
    from .beit import *
  File "/usr/local/lib/python3.6/dist-packages/timm/models/beit.py", line 33, in <module>
    from .vision_transformer import 

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


In [ ]:
import numpy as np
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = predictor.predict(images.numpy())
        _, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
estimator.delete_endpoint()